# 載入資料

In [1]:
import matplotlib.pyplot as plt
import os
import keras

os.environ['CUDA_VISIBLE_DEVICES'] = ""
data_path = "kaggle_dogcat/train"
test_path = "kaggle_dogcat/test"

row_size = 300
column_size = 300
batch_size = 10
# 建立image generator
train_gene = keras.preprocessing.image.ImageDataGenerator(rotation_range=30,
                                                            width_shift_range=0.1,
                                                            height_shift_range=0.1,
                                                            rescale=1/255,
                                                            horizontal_flip=True)

# 額外產生圖片
train_generator = train_gene.flow_from_directory(data_path,
                                                target_size=(row_size, column_size),
                                                batch_size=batch_size)


'''
for i in range(9):
    plt.subplot(3,3,i+1)
    img = train_generator.next()
    print(img[1])
    plt.imshow(img[0][0])
plt.show()
'''


Using TensorFlow backend.
c:\users\lclee\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\lclee\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\lclee\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

Found 4000 images belonging to 2 classes.


'\nfor i in range(9):\n    plt.subplot(3,3,i+1)\n    img = train_generator.next()\n    print(img[1])\n    plt.imshow(img[0][0])\nplt.show()\n'

# 重新整理測試資料

In [2]:
# 整理測試資料
import cv2
import numpy as np

data_path = "kaggle_dogcat/test/test"

x_test = []
file_list = os.listdir(data_path)
for data in file_list:
    img = cv2.imread(os.path.join(data_path,data))
    res = cv2.resize(img, dsize=(row_size, column_size), interpolation=cv2.INTER_CUBIC)
    x_test.append(res/255)

x_test = np.array(x_test)
print(x_test.shape)

(400, 300, 300, 3)


# 建立網路

In [3]:
def cnn_model():
    #keras.layers.BatchNormalization(),
    layer_list = [keras.layers.Conv2D(filters=10,kernel_size=(15,15),
                                      input_shape=(row_size,column_size,3),
                                      activation='relu'),
                  keras.layers.BatchNormalization(),
                  keras.layers.MaxPooling2D((2,2)),
                  keras.layers.Conv2D(filters=20,kernel_size=(10,10),activation='relu'),
                  keras.layers.BatchNormalization(),
                  keras.layers.MaxPooling2D((2,2)),
                  keras.layers.Conv2D(filters=30,kernel_size=(5,5),activation='relu'),
                  keras.layers.BatchNormalization(),
                  keras.layers.MaxPooling2D((2,2)),
                  keras.layers.Conv2D(filters=40,kernel_size=(3,3),activation='relu'),
                  keras.layers.BatchNormalization(),
                  keras.layers.MaxPooling2D((2,2)),
                  keras.layers.Flatten(),
                  keras.layers.Dense(units=100,activation='relu'),
                  keras.layers.Dense(units=2,activation='softmax')]
    
    model = keras.models.Sequential(layer_list)
    return model

cnn_model = cnn_model()
cnn_model.summary()
cnn_model.compile(loss="categorical_crossentropy",
             optimizer=keras.optimizers.Adam(),
             metrics=['accuracy'])


cnn_model.fit_generator(train_generator,
                        epochs=25,
                        steps_per_epoch=1000,
                       shuffle=True)


W0908 00:16:56.018635  6988 deprecation_wrapper.py:119] From c:\users\lclee\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0908 00:16:56.030602  6988 deprecation_wrapper.py:119] From c:\users\lclee\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0908 00:16:56.032598  6988 deprecation_wrapper.py:119] From c:\users\lclee\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0908 00:16:56.050549  6988 deprecation_wrapper.py:119] From c:\users\lclee\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 286, 286, 10)      6760      
_________________________________________________________________
batch_normalization_1 (Batch (None, 286, 286, 10)      40        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 143, 143, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 134, 134, 20)      20020     
_________________________________________________________________
batch_normalization_2 (Batch (None, 134, 134, 20)      80        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 67, 67, 20)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 63, 63, 30)        15030     
__________

In [4]:
import pandas as pd

pred = cnn_model.predict(np.array(x_test))[:,1]
output_df = pd.DataFrame([str(i).zfill(3) for i in range(len(pred))])
output_df.columns = ['ID']
output_df['Predicted'] = pred
output_df.to_csv('res.csv',index=False)
output_df.head(20)

,ID,Predicted
0,000,0.087256
1,001,0.723323
2,002,0.999726
3,003,0.012620
4,004,0.829181
5,005,0.961019
6,006,0.148309
7,007,0.978520
8,008,0.993866
9,009,0.998451
